# Генератор обучающей выборки (песок)

Равномерное заполнение изображения 100x100 px "песчинками" - кругами разного радиуса

In [1]:
import numpy as np
from PIL import Image, ImageDraw
from numpy.random import randint, randn, poisson, choice
from numpy import pi
from tqdm import tqdm_notebook, tnrange

### Гиперпараметры

In [65]:
# размеры изображения обучающей выборки
W = 256
H = 256
# коэффициент антиалиасинга
AA = 3
# линейный тренд l = lambda = Kx + b
lStart = 0.2
lEnd = 5
K = (lEnd - lStart) / (W * AA)
# процент "обхождения x-ов"
B = 0.3
# радиус "песчинок"
r = 3
# названия и путь к файлам
fileName = "sample"
ext = ".jpg"
filePath = "../data/sand/trend2/"

### Функции генерирования сэмплов

In [53]:
def generateSample(w, h, l, B, num, AA, val=False, side=1):
    # новое ч/б изображение, размером в AA**2 раз больше для суперсэмплинга
    W = w * AA
    H = h * AA
    R = r * AA
    image = Image.new('1', (W, H), 'white')
    draw = ImageDraw.Draw(image)
    # карта заполнения, чтобы предотвратить взаимопроникновение "песчинок"
    pixelMap = image.load()
    # цикл по x-ам
    for k in range(int(B * W)):
        x = randint(1, W)
        # число событий пуассоновского потока на данной вертикали
        Nx = poisson(l(x))
        # находим недоступные на данный момент y
        bannedY = set()
        for y in range(H):
            banCond = 0
            for j in range(R + 1):
                # условия недоступности y
                if (x - j > 0):
                    cLeft = pixelMap[x - j, y]
                else:
                    cLeft = True
                if (x + j < W - 1):
                    cRight = pixelMap[x + j, y]
                else:
                    cRight = True
                if (y - j > 0):
                    cTop = pixelMap[x, y - j]
                else:
                    cTop = True
                if (y + j < H - 1):
                    cBot = pixelMap[x, y + j]
                else:
                    cBot = True
                freeCond = banCond or cLeft or cRight or cTop or cBot
            if(not(freeCond)):
                bannedY.add(y)
        # заполняем вертикаль
        freeY = set(range(W)) - bannedY
        for j in range(Nx):
            # генерируем y из доступных значений
            y = choice(list(freeY))
            draw.ellipse((x - R, y - R, x + R, y + R), fill='black')
            freeY -= set(range(y - R, y + R + 1))
    # суперсэмплинг с антиалиасингом
    image = image.resize((w, h))
    fullFilePath = filePath
    if (val == False):
        fullFilePath += "train/"
    else:
        fullFilePath += "validation/"
    if (side == 1):
        fullFilePath += "side1/"
    elif (side == 2):
        fullFilePath += "side2/"
    else:
        fullFilePath += "panorama/"
    fullFilePath += (fileName + str(num) + ext)
    image.save(fullFilePath)

In [66]:
# количество сэмплов для генерации
N = 20
# train/validation = 80/20
N_train = int(0.8 * N)
N_val = N - N_train
for i in tnrange(N_train, desc='Train'):
    generateSample(W, H, lambda x: lStart, B, i, AA, val=False, side=1)
    generateSample(W, H, lambda x: lEnd, B, i, AA, val=False, side=2)
    generateSample(W, H, lambda x: lStart + K * x, B, i, AA, val=False, side=3)

for i in tnrange(N_val, desc='Validation'):
    generateSample(W, H, lambda x: lStart, B, i, AA, val=False, side=1)
    generateSample(W, H, lambda x: lEnd, B, i, AA, val=False, side=2)
    generateSample(W, H, lambda x: lStart + K * x, B, i, AA, val=False, side=3)

In [51]:
image = Image.new('1', (10, 10), 'white')
draw = ImageDraw.Draw(image)

In [52]:
draw.ellipse((5 - 1, 5 - 1, 5 + 1, 5 + 1), fill='black')

In [55]:
px = image.load()

In [48]:
l = lambda x: 5

In [50]:
l(88)

5

In [5]:
print(a)

{1, 2, 4, 5}


In [118]:
a = set([1,2,3,4])

In [119]:
b = set([4, 5, 6, 7])

In [121]:
b - a

{5, 6, 7}

In [105]:
b(1)

TypeError: 'set' object is not callable

In [93]:
a.add(5)

In [106]:
b

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [117]:
b - set([1])

{0, 2, 3, 4, 5, 6, 7, 8, 9}